In [ ]:
import pandas as pd
import json
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
print(pd.__version__)
df = pd.read_json('radical/analyse_fase-kenmerken_onderzoek/Data/bs-posts-2-clean.jsonl.gz', orient='records', lines=True, compression='gzip')
df.head()



model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:

def predict_in_batches(texts, batch_size=32):
    all_labels = []
    all_confidences = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = model(**inputs)

        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)

        sentiment_map = {
            0: "Very Negative",
            1: "Negative",
            2: "Neutral",
            3: "Positive",
            4: "Very Positive"
        }

        for p in probs:
            idx = torch.argmax(p).item()
            all_labels.append(sentiment_map[idx])
            all_confidences.append(float(p[idx]))

    return all_labels, all_confidences


In [ ]:
labels, confs = predict_in_batches(df["content.text"].astype(str).tolist(), batch_size=16)

df["sentiment"] = labels
df["confidence"] = confs

#exporteren van gelabelde dataset
df.to_json('radical/analyse_fase-kenmerken_onderzoek/Data/bs-posts2-sentiment-labeld.jsonl.gz', orient='records', lines=True, compression='gzip')